<a href="https://colab.research.google.com/github/GRZ321/emprestimo_banco/blob/main/projeto_banco.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd 

In [2]:
credit = pd.read_csv('/content/credit_scoring_eng.csv')
credit.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,Secondary Education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding


In [3]:
credit.dtypes

children              int64
days_employed       float64
dob_years             int64
education            object
education_id          int64
family_status        object
family_status_id      int64
gender               object
income_type          object
debt                  int64
total_income        float64
purpose              object
dtype: object

In [4]:
# identificando que as mesmas linhas ausentes de days employed correspondem ao total_income
days_employed_na = credit.loc[credit['days_employed'].isna()]
days_employed_na.loc[days_employed_na['total_income'].isna()]


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
12,0,NaN,65,secondary education,1,civil partnership,1,M,retiree,0,NaN,to have a wedding
26,0,NaN,41,secondary education,1,married,0,M,civil servant,0,NaN,education
29,0,NaN,63,secondary education,1,unmarried,4,F,retiree,0,NaN,building a real estate
41,0,NaN,50,secondary education,1,married,0,F,civil servant,0,NaN,second-hand car purchase
55,0,NaN,54,secondary education,1,civil partnership,1,F,retiree,1,NaN,to have a wedding
...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Secondary Education,1,married,0,M,business,0,NaN,purchase of a car
21495,1,NaN,50,secondary education,1,civil partnership,1,F,employee,0,NaN,wedding ceremony
21497,0,NaN,48,BACHELOR'S DEGREE,0,married,0,F,business,0,NaN,building a property
21502,1,NaN,42,secondary education,1,married,0,F,employee,0,NaN,building a real estate


Eu identifiquei que as linhas do days_employed que estão com valores ausentes são as mesmas linhas que também tem valores ausentes em total_income. Isso pode ter sido erro de sistema, parece que essas duas colunas são interligadas. Irei colocar zero em days_employed e total_income. Pois acredito que se excluir todas as linhas ausentes, irei perder muitos dados.

In [5]:
# trocando por zero as linhas ausentes
credit['days_employed'] = credit['days_employed'].fillna(0)
credit['total_income'] = credit['total_income'].fillna(0)
credit.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

Procurando por **duplicatas**

In [6]:
# valores duplicados encontrados na coluna de education
credit['education'].value_counts()


secondary education    13750
bachelor's degree       4718
SECONDARY EDUCATION      772
Secondary Education      711
some college             668
BACHELOR'S DEGREE        274
Bachelor's Degree        268
primary education        250
Some College              47
SOME COLLEGE              29
PRIMARY EDUCATION         17
Primary Education         15
graduate degree            4
Graduate Degree            1
GRADUATE DEGREE            1
Name: education, dtype: int64

In [7]:
# Retirei todas as linhas duplicadas apenas colocando todas as linhas em minusculo
credit['education'] = credit['education'].str.lower()

#verificando como ficou
credit.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,-4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,-5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,-4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding


In [8]:
# Achei mais facil aplicar a função abs que dá o numero absoluto do numero. sem o sinal.
credit['days_employed'] = abs( credit['days_employed'] )

In [9]:
# verificando como está as coisas 
credit.head()


,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,8437.673028,42,bachelor's degree,0,married,0,F,employee,0,40620.102,purchase of the house
1,1,4024.803754,36,secondary education,1,married,0,F,employee,0,17932.802,car purchase
2,0,5623.422610,33,secondary education,1,married,0,M,employee,0,23341.752,purchase of the house
3,3,4124.747207,32,secondary education,1,married,0,M,employee,0,42820.568,supplementary education
4,0,340266.072047,53,secondary education,1,civil partnership,1,F,retiree,0,25378.572,to have a wedding


In [10]:
# Encontrei um valor negativo em quantidade de filhos, irei fazer o mesmo que fiz nos days_employed
credit['children'].value_counts()

credit['children'] = abs(credit['children'])

In [11]:
credit['children'].value_counts()

0     14149
1      4865
2      2055
3       330
20       76
4        41
5         9
Name: children, dtype: int64

### Primeira Questão: Existe alguma relação entre ter filhos e pagar um empréstimo em dia?


Resposta: Aparentemente, a relação é inversamente proporcional, ou seja, quanto mais filhos se tem, maiores as probabilidades de quitar o empréstimo.

In [12]:
credit.pivot_table(values='debt', columns=['children'], aggfunc=sum)

children,0,1,2,3,4,5,20
debt,1063,445,194,27,4,0,8


### Segunda questão: Existe alguma relação entre o estado civil e o pagamento de um empréstimo no prazo estipulado?

Resposta: Pessoas casadas ou em parceria cívil  tendem a fazer mais dívidas do que o restante das pessoas.

In [13]:
credit.pivot_table(values='debt', columns='family_status', aggfunc=sum)

family_status,civil partnership,divorced,married,unmarried,widow / widower
debt,388,85,931,274,63


## Terceira Questão: Existe uma relação entre o nível de renda e o pagamento de um empréstimo no prazo?
